In [1]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import pickle
import json

In [2]:
def get_data(self, features=['DOI', 'Study.Title.O']):
    if features == 'all':
        self.df = pd.read_excel('data/new_data.xlsx', encoding='ansi',)
    else:
        self.df = pd.read_excel('data/new_data.xlsx', encoding='ansi', usecols=features)

In [3]:
def __graph_wos__(self, g, DOIS, hops):
    if hops == 0:
        return
    print('---------Running for {} hop---------'.format(hops))
    new_dois = set()
    for idx_doi, doi in enumerate(DOIS):
        name = doi.replace('/', '_')
        file_name = self.path_head + '/{}/citations_{}.txt'.format(name, name)
        if os.path.exists(file_name):
            print('References are getting fetched for: ', idx_doi, name)
            df_doi = pd.read_csv(file_name, sep='\t', lineterminator='\r', encoding="utf-16le",
                                 index_col=False, quotechar=None, quoting=3, usecols=['DI'])
            df_doi = df_doi.dropna()
            for i, citation_row in df_doi.iterrows():
                if not g.has_edge(name, citation_row['DI']):
                    new_dois.add(citation_row['DI'])
                    g.add_edge(name, citation_row['DI'])
    print('Number of DOIs for next hop ', len(new_dois))
    self.__graph_wos__(g, new_dois, hops - 1)


In [4]:
def graph_wos(self, hops=2):
    # Based on WOS data
    g = nx.DiGraph()
    self.df.dropna(subset=['DOI'])
    self.__graph_wos__(g, set(self.df['DOI']), hops)
    print('---------Getting betweenness---------')
    betweenness_dict = nx.betweenness_centrality(g)
    nx.set_node_attributes(g, betweenness_dict, 'betweenness')
    print('---------Getting Degree---------')
    degree_dict = dict(g.degree(g.nodes()))
    nx.set_node_attributes(g, degree_dict, 'degree')
# 
    print('-------------Saving Graph--------------')
    print('Number of nodes in the graph: ', len(g.nodes()))
    print('Number of edges in the graph: ', len(g.edges()))
    nx.write_gpickle(g, 'data/references_network_{}hops_wos.gpickle'.format(hops))
    nx.write_gexf(g, "data/references_network_{}hops_wos.gexf".format(hops))
    print('-------------Graph Saved--------------')

In [5]:
def __graph_mag__(self, g, IDS, df_citations, graph_type, hops):
    if hops == 0:
        return
    print('---------Running for {} hop---------'.format(hops))
    new_IDS = []
    for idx, cur_id in enumerate(IDS):
        if graph_type == 'references':
            print('References are getting fetched for: ', idx, ' ', cur_id)
            new_nodes = list(df_citations.loc[df_citations['from'] == cur_id]['to'])
        elif graph_type == 'citations':
            print('Citations are getting fetched for: ', idx, ' ', cur_id)
            new_nodes = list(df_citations.loc[df_citations['to'] == cur_id]['from'])
        for edge_node in new_nodes:
            if not g.has_edge(cur_id, edge_node):
                g.add_edge(cur_id, edge_node)
        new_IDS.extend(new_nodes)
    print('Number of DOIs for next hop ', len(new_IDS))
    self.__graph_mag__(g, set(new_IDS), df_citations, graph_type, hops - 1)

In [6]:
def graph_mag(self, graph_type='references', hops=2):
    # Based on MAG data
    g = nx.DiGraph()
    self.df.dropna(subset=['DOI'])
    print('---------Getting all the edges---------')
    df_citations = pd.read_csv("data/MAG_data/psychology_citations.csv")
    # map all the dois to an id and send that
    IDS = dict()
    mappings = json.loads(json.dumps(pickle.load(open("data/MAG_data/rpp.pkl", "rb"))))
    for doi in list(self.df['DOI']):
        if doi in mappings:
            IDS[mappings[doi][0]] = doi
    self.__graph_mag__(g, IDS.keys(), df_citations, graph_type, 2)
    print('---------Getting betweenness---------')
    betweenness_dict = nx.betweenness_centrality(g)
    nx.set_node_attributes(g, betweenness_dict, 'betweenness')
    print('---------Getting Degree---------')
    degree_dict = dict(g.degree(g.nodes()))
    nx.set_node_attributes(g, degree_dict, 'degree')

    print('-------------Saving Graph--------------')
    print('Number of nodes in the graph: ', len(g.nodes()))
    print('Number of edges in the graph: ', len(g.edges()))
    nx.write_gpickle(g, 'data/{}_network_{}hops_mag.gpickle'.format(graph_type, hops))
    nx.write_gexf(g, "data/{}_network_{}hops_mag.gexf".format(graph_type, hops))
    pickle.dump(IDS, open('data/IDS_doi_mapping.pkl', 'wb'))
    print('-------------Graph Saved--------------')

In [7]:
def display_graph(self, file):
    print('-------------Generating Graph----------')
    g = nx.read_gpickle(file)
    print('Number of nodes in the graph: ', len(g.nodes()))
    print('Number of edges in the graph: ', len(g.edges()))
    nx.draw_networkx(G=g, pos=nx.spring_layout(g), node_color='r', alpha=0.8,
                     node_size=[g.degree(n) * 3 for n in g.nodes()], with_labels=False)
    plt.show()

In [8]:

class NetworkGraph:
    def __init__(self):
        self.path_head = '../DataExtraction/WOS/RPPdataConverted'
        self.df = None
    
    get_data = get_data
    __graph_wos__ = __graph_wos__
    graph_wos = graph_wos
    __graph_mag__ = __graph_mag__
    graph_mag = graph_mag
    display_graph = display_graph
    

In [9]:
if __name__ == '__main__':
    cnn = NetworkGraph()
    
    cnn.get_data()
    # WOS data has only option of getting references. So only option we can change is number of hops
#     cnn.graph_wos(hops=2)
    
    # Mag data has two options for generating the graph_type='references' and 'citations', Also we can set the number of hops
    cnn.graph_mag(graph_type='citations', hops=2)

---------Getting all the edges---------
---------Running for 2 hop---------
Citations are getting fetched for:  0   2150344675
Citations are getting fetched for:  1   2082803587
Citations are getting fetched for:  2   2024024483
Citations are getting fetched for:  3   2107533858
Citations are getting fetched for:  4   2094310115
Citations are getting fetched for:  5   2024811810
Citations are getting fetched for:  6   2005429355
Citations are getting fetched for:  7   1964385890
Citations are getting fetched for:  8   2049267616
Citations are getting fetched for:  9   2114244727
Citations are getting fetched for:  10   2009908974
Citations are getting fetched for:  11   2044434223
Citations are getting fetched for:  12   2075830827
Citations are getting fetched for:  13   2019587681
Citations are getting fetched for:  14   2129976542
Citations are getting fetched for:  15   2134384538
Citations are getting fetched for:  16   2039880978
Citations are getting fetched for:  17   212145467

Citations are getting fetched for:  87   2023555571
Citations are getting fetched for:  88   2884665844
Citations are getting fetched for:  89   2168029685
Citations are getting fetched for:  90   2158428665
Citations are getting fetched for:  91   2079326716
Citations are getting fetched for:  92   2131689986
Citations are getting fetched for:  93   2487943687
Citations are getting fetched for:  94   2167079436
Citations are getting fetched for:  95   1705673232
Citations are getting fetched for:  96   2111341078
Citations are getting fetched for:  97   2758804000
Citations are getting fetched for:  98   2795078176
Citations are getting fetched for:  99   2085585445
Citations are getting fetched for:  100   2724725290
Citations are getting fetched for:  101   2619408941
Citations are getting fetched for:  102   2094400047
Citations are getting fetched for:  103   2572845625
Citations are getting fetched for:  104   2564162106
Citations are getting fetched for:  105   2586837565
Citati

Citations are getting fetched for:  243   1904411994
Citations are getting fetched for:  244   60523869
Citations are getting fetched for:  245   2337015143
Citations are getting fetched for:  246   2952332649
Citations are getting fetched for:  247   2128872811
Citations are getting fetched for:  248   2894628209
Citations are getting fetched for:  249   2174256498
Citations are getting fetched for:  250   2047640947
Citations are getting fetched for:  251   201327993
Citations are getting fetched for:  252   2277049726
Citations are getting fetched for:  253   1995081088
Citations are getting fetched for:  254   2770535808
Citations are getting fetched for:  255   2888402304
Citations are getting fetched for:  256   2285503876
Citations are getting fetched for:  257   2516059544
Citations are getting fetched for:  258   2891711897
Citations are getting fetched for:  259   2130249119
Citations are getting fetched for:  260   2140505504
Citations are getting fetched for:  261   2886108

Citations are getting fetched for:  398   2061502645
Citations are getting fetched for:  399   2144995510
Citations are getting fetched for:  400   1741228216
Citations are getting fetched for:  401   830310592
Citations are getting fetched for:  402   1992427716
Citations are getting fetched for:  403   1982925002
Citations are getting fetched for:  404   2763622607
Citations are getting fetched for:  405   1966573780
Citations are getting fetched for:  406   2325285077
Citations are getting fetched for:  407   1552746713
Citations are getting fetched for:  408   2512652507
Citations are getting fetched for:  409   816908511
Citations are getting fetched for:  410   2017396962
Citations are getting fetched for:  411   2126645475
Citations are getting fetched for:  412   2958067950
Citations are getting fetched for:  413   2020149497
Citations are getting fetched for:  414   2108360963
Citations are getting fetched for:  415   1994688775
Citations are getting fetched for:  416   288640

Citations are getting fetched for:  554   2783873975
Citations are getting fetched for:  555   2070842295
Citations are getting fetched for:  556   1188334521
Citations are getting fetched for:  557   2559970240
Citations are getting fetched for:  558   1989807040
Citations are getting fetched for:  559   2544667591
Citations are getting fetched for:  560   2112392142
Citations are getting fetched for:  561   2132151248
Citations are getting fetched for:  562   2321222613
Citations are getting fetched for:  563   2512194521
Citations are getting fetched for:  564   2789706713
Citations are getting fetched for:  565   2510818267
Citations are getting fetched for:  566   1987185633
Citations are getting fetched for:  567   2070580194
Citations are getting fetched for:  568   2141981670
Citations are getting fetched for:  569   1971948521
Citations are getting fetched for:  570   2052885486
Citations are getting fetched for:  571   2142604275
Citations are getting fetched for:  572   1832

...

Citations are getting fetched for:  5437   2349003448
Citations are getting fetched for:  5438   2181624505
Citations are getting fetched for:  5439   2290971321
Citations are getting fetched for:  5440   2097181372
Citations are getting fetched for:  5441   2051437248
Citations are getting fetched for:  5442   2605085377
Citations are getting fetched for:  5443   2810180293
Citations are getting fetched for:  5444   2095772362
Citations are getting fetched for:  5445   2193126097
Citations are getting fetched for:  5446   2098492120
Citations are getting fetched for:  5447   2128736989
Citations are getting fetched for:  5448   1560244959
Citations are getting fetched for:  5449   2516447969
Citations are getting fetched for:  5450   2558128868
Citations are getting fetched for:  5451   2157540070
Citations are getting fetched for:  5452   1861022440
Citations are getting fetched for:  5453   1918104299
Citations are getting fetched for:  5454   2467394288
Citations are getting fetche

Citations are getting fetched for:  5589   2785113552
Citations are getting fetched for:  5590   2994959830
Citations are getting fetched for:  5591   2110158305
Citations are getting fetched for:  5592   2741204450
Citations are getting fetched for:  5593   2077586916
Citations are getting fetched for:  5594   2128737773
Citations are getting fetched for:  5595   2055239151
Citations are getting fetched for:  5596   2045244915
Citations are getting fetched for:  5597   2995844597
Citations are getting fetched for:  5598   2768434687
Citations are getting fetched for:  5599   2571925001
Citations are getting fetched for:  5600   1970173454
Citations are getting fetched for:  5601   2026993166
Citations are getting fetched for:  5602   2770335247
Citations are getting fetched for:  5603   2415392279
Citations are getting fetched for:  5604   2128901661
Citations are getting fetched for:  5605   2921494055
Citations are getting fetched for:  5606   1978332714
Citations are getting fetche

Citations are getting fetched for:  5741   2023258370
Citations are getting fetched for:  5742   2159802630
Citations are getting fetched for:  5743   2229696778
Citations are getting fetched for:  5744   2789767434
Citations are getting fetched for:  5745   2001565969
Citations are getting fetched for:  5746   2103179541
Citations are getting fetched for:  5747   2187786519
Citations are getting fetched for:  5748   2116385047
Citations are getting fetched for:  5749   369981725
Citations are getting fetched for:  5750   2521379
Citations are getting fetched for:  5751   2185722158
Citations are getting fetched for:  5752   2069428529
Citations are getting fetched for:  5753   2783344952
Citations are getting fetched for:  5754   1977776446
Citations are getting fetched for:  5755   2098198852
Citations are getting fetched for:  5756   1995307336
Citations are getting fetched for:  5757   2001303880
Citations are getting fetched for:  5758   2263480660
Citations are getting fetched fo

Citations are getting fetched for:  5893   2889972836
Citations are getting fetched for:  5894   2994109541
Citations are getting fetched for:  5895   2902260836
Citations are getting fetched for:  5896   2010348649
Citations are getting fetched for:  5897   2171763819
Citations are getting fetched for:  5898   1933376620
Citations are getting fetched for:  5899   1893858414
Citations are getting fetched for:  5900   2046950511
Citations are getting fetched for:  5901   2739371121
Citations are getting fetched for:  5902   2361654386
Citations are getting fetched for:  5903   2560523382
Citations are getting fetched for:  5904   2543811703
Citations are getting fetched for:  5905   2790882426
Citations are getting fetched for:  5906   2000256123
Citations are getting fetched for:  5907   2586410116
Citations are getting fetched for:  5908   2111995017
Citations are getting fetched for:  5909   2492300435
Citations are getting fetched for:  5910   2562915483
Citations are getting fetche

Citations are getting fetched for:  6045   2133622627
Citations are getting fetched for:  6046   2594602854
Citations are getting fetched for:  6047   2605678444
Citations are getting fetched for:  6048   2102034286
Citations are getting fetched for:  6049   2605350770
Citations are getting fetched for:  6050   57147253
Citations are getting fetched for:  6051   2553741173
Citations are getting fetched for:  6052   2079719291
Citations are getting fetched for:  6053   1995997052
Citations are getting fetched for:  6054   2588901247
Citations are getting fetched for:  6055   1997045645
Citations are getting fetched for:  6056   2014576530
Citations are getting fetched for:  6057   2108489623
Citations are getting fetched for:  6058   2028142492
Citations are getting fetched for:  6059   2572582813
Citations are getting fetched for:  6060   2587819948
Citations are getting fetched for:  6061   2607972272
Citations are getting fetched for:  6062   2969141172
Citations are getting fetched 